In [5]:
import pandas as pd
import numpy as np

# Carregar o arquivo Excel
xls = pd.ExcelFile('Exercicio_Python_CRM .xlsx')

# Obter os nomes das abas
sheet_names = xls.sheet_names

# Para cada aba, ler o schema e as primeiras 10 linhas
for sheet in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, nrows=10)
    print(f"Schema para {sheet}:")
    print(df.dtypes)
    print(f"Primeiras 10 linhas para {sheet}:")
    print(df.head(10))

Schema para Instruções:
Unnamed: 0    float64
Unnamed: 1     object
dtype: object
Primeiras 10 linhas para Instruções:
   Unnamed: 0                                         Unnamed: 1
0         NaN                    Teste de Seleção - Python CRM 2
1         NaN                                                NaN
2         NaN  Neste arquivo você econtra dados hipotéticos d...
3         NaN                                                NaN
4         NaN                                        Instruções:
5         NaN                                                NaN
6         NaN  Utilizando Pandas e Numpy (ou se preferir, out...
7         NaN                                                NaN
8         NaN  0. Importar os dados das abas em cor verde do ...
9         NaN                                                NaN
Schema para Transações:
ID_Trans                     int64
Data                datetime64[ns]
ID_Loja                      int64
ID_Produto                   int64
ID

In [6]:
import openpyxl

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de transações
ws = wb['Transações']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_t = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_t.columns:
    # Se o tipo de dados da coluna é object (string)
    if df_t[col].dtype == 'object':
        # Remova os espaços em branco no início e no final
        df_t[col] = df_t[col].str.strip()

In [7]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7791 entries, 0 to 7790
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_Trans          7791 non-null   int64         
 1   Data              7791 non-null   datetime64[ns]
 2   ID_Loja           7791 non-null   int64         
 3   ID_Produto        7791 non-null   int64         
 4   ID_Cliente        7791 non-null   int64         
 5   UnidadesVendidas  7791 non-null   int64         
 6   ValorTotal        7791 non-null   float64       
 7   Promo             7775 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 487.1 KB


In [8]:
df_t['Promo'] = df_t['Promo'].astype(str)

In [9]:
df_t.head()

,ID_Trans,Data,ID_Loja,ID_Produto,ID_Cliente,UnidadesVendidas,ValorTotal,Promo
0,1,2022-01-01,7,123005,90023,2,99.98,0.0
1,2,2022-01-01,9,123004,90092,1,35.99,0.0
2,3,2022-01-01,1,123001,90027,2,51.98,0.0
3,4,2022-01-01,3,123001,90097,2,51.98,0.0
4,5,2022-01-01,4,123004,90089,2,71.98,0.0


In [10]:
df_t['Data'].min(), df_t['Data'].max()

(Timestamp('2022-01-01 00:00:00'), Timestamp('2023-09-30 00:00:00'))

In [11]:
# Filtrando colunas que não contêm 'ID' no nome
cols = [col for col in df_t.columns if 'ID' not in col]

# Descrevendo apenas essas colunas
df_t[cols].describe()

,UnidadesVendidas,ValorTotal
count,7791.000000,7791.000000
mean,2.302657,114.133276
std,0.892296,72.627427
min,1.000000,16.990000
25%,2.000000,55.990000
50%,2.000000,89.990000
75%,3.000000,161.973000
max,4.000000,367.960000


In [12]:
# verificando valores nulos
null_counts = df_t.isnull().sum()
print(null_counts)

ID_Trans            0
Data                0
ID_Loja             0
ID_Produto          0
ID_Cliente          0
UnidadesVendidas    0
ValorTotal          0
Promo               0
dtype: int64


In [13]:
# verificando valores duplicados

for column in df_t.columns:
    print(f"\nColumn: {column}")
    print(df_t[column].value_counts(dropna=False))


Column: ID_Trans
1       1
5175    1
5203    1
5202    1
5201    1
       ..
2593    1
2592    1
2591    1
2590    1
7791    1
Name: ID_Trans, Length: 7791, dtype: int64

Column: Data
2022-03-17    88
2022-04-27    84
2022-05-14    82
2023-03-11    77
2022-03-09    75
              ..
2022-07-14     1
2022-07-13     1
2023-08-02     1
2023-06-11     1
2022-05-23     1
Name: Data, Length: 546, dtype: int64

Column: ID_Loja
3     927
9     903
2     884
4     862
7     853
6     842
8     837
5     813
10    437
1     433
Name: ID_Loja, dtype: int64

Column: ID_Produto
123004    850
123005    842
123003    828
123002    824
123008    822
123006    806
123007    795
123001    639
123009    592
123010    380
123000    228
123011    185
Name: ID_Produto, dtype: int64

Column: ID_Cliente
90016    100
90096     99
90083     99
90011     98
90089     96
        ... 
90042     58
90010     57
90020     50
90000     39
90099     32
Name: ID_Cliente, Length: 100, dtype: int64

Column: UnidadesVe

##### Primeira parte da análise:
- análise descritiva da base de Transações
- agregação Ano e Mês
- KPI Ticket Médio
###### nesta primeira parte da análise é possível verificar que:
- a base apenas possui dados para 01-2022 até 09-2022 e depois 01-2023 até 09-2023 faltando o último trimestre dos dois anos; época de Natal!
- as vendas de 2023 apenas se recuperaram no terceiro trimestre
- o ano de 2023 teve menos promoções e menos Unidades vendidas em relação a 2022, porém obteve um Valor Total maior indicando que houve um Ticket Médio maior em 2023 em relação a 2022
- o Ticket Médio de 2023 foi sempre maior que 2022 no agregado mês a mês
 

In [14]:
import plotly.graph_objects as go

# Agrupando os dados por data
grouped = df_t.groupby('Data').sum()

# Criando o gráfico de barras para 'ValorTotal'
bar1 = go.Bar(x=grouped.index, y=grouped['ValorTotal'], name='ValorTotal')

# Criando outro gráfico de barras para 'UnidadesVendidas'
bar2 = go.Bar(x=grouped.index, y=grouped['UnidadesVendidas'], name='UnidadesVendidas', yaxis='y2')

# Criando a figura e adicionando os gráficos
fig = go.Figure(data=[bar1, bar2])

# Configurando os eixos
fig.update_layout(
    yaxis=dict(title='ValorTotal'),
    yaxis2=dict(title='UnidadesVendidas', overlaying='y', side='right')
)

fig.show()

In [15]:
import seaborn as sns

# Data to datetime
df_t['Data'] = pd.to_datetime(df_t['Data'])

# Criando colunas para o mês e o ano
df_t['Year'] = df_t['Data'].dt.year
df_t['Month'] = df_t['Data'].dt.month

# Agrupando os dados por ano e mês
grouped = df_t.groupby(['Year', 'Month']).sum().reset_index()

# Obtendo as cores da paleta "Pastel2"
colors = sns.color_palette("Pastel2", len(grouped['Year'].unique())).as_hex()

# Criando o gráfico de barras para 'ValorTotal'
fig = go.Figure()

for i, year in enumerate(grouped['Year'].unique()):
    fig.add_trace(go.Bar(x=grouped[grouped['Year'] == year]['Month'], 
                         y=grouped[grouped['Year'] == year]['ValorTotal'], 
                         name=str(year),
                         marker_color=colors[i]))  # Aplicando as cores

# Configurando os eixos
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='ValorTotal'),
)

fig.show()

In [16]:
# Definindo a paleta de cores Pastel2
pastel2 = ['#b3e2cd', '#fdcdac', '#cbd5e8', '#f4cae4', '#e6f5c9', '#fff2ae', '#f1e2cc', '#cccccc']

# Criando colunas para o mês e o ano
df_t['Year'] = df_t['Data'].dt.year
df_t['Month'] = df_t['Data'].dt.month

# Agrupando os dados por ano e mês
grouped = df_t.groupby(['Year', 'Month']).sum().reset_index()

# Criando a figura
fig = go.Figure()

# Adicionando o gráfico de barras para 'ValorTotal'
for i, year in enumerate(grouped['Year'].unique()):
    fig.add_trace(go.Bar(x=grouped[grouped['Year'] == year]['Month'], 
                         y=grouped[grouped['Year'] == year]['ValorTotal'], 
                         name='ValorTotal ' + str(year),
                         marker_color=pastel2[i % len(pastel2)]))  # Use a cor correspondente da paleta

# Adicionando o gráfico de linha para 'UnidadesVendidas'
for i, year in enumerate(grouped['Year'].unique()):
    line_color = '#ff0000' if year == 2023 else pastel2[(i + len(pastel2) // 2) % len(pastel2)]  # Use a cor vermelha para 2023
    fig.add_trace(go.Scatter(x=grouped[grouped['Year'] == year]['Month'], 
                             y=grouped[grouped['Year'] == year]['UnidadesVendidas'], 
                             name='UnidadesVendidas ' + str(year), 
                             yaxis='y2',
                             line_color=line_color))

# Configurando os eixos e a legenda
fig.update_layout(
    title={
        'text': "Unidades Vendidas e Valor Total por Mês e Ano",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
    xaxis=dict(title='Month'),
    yaxis=dict(title='ValorTotal'),
    yaxis2=dict(title='UnidadesVendidas', overlaying='y', side='right'),
    autosize=False,
    width=900,  # Diminui a largura do gráfico em 10%
    legend=dict(
        x=1.1,  # Posiciona a legenda mais para a direita
        y=1,  # Posiciona a legenda no topo
        bgcolor='rgba(255, 255, 255, 0)',  # Fundo transparente para a legenda
        bordercolor='rgba(255, 255, 255, 0)'  # Borda transparente para a legenda
    )
)

fig.show()

In [17]:
# Preenchendo os valores nulos na coluna 'Promo' com 0
df_t['Promo'] = df_t['Promo'].fillna(0)

# Convertendo a coluna 'Promo' para float, substituindo NaNs por 0 e depois convertendo para inteiro
df_t['Promo'] = np.nan_to_num(df_t['Promo'].astype(float)).astype(int)

# Filtrando os dados para os anos de 2022 e 2023
df_t['Year'] = df_t['Data'].dt.year
df_t['Month'] = df_t['Data'].dt.month
df_filtered = df_t[df_t['Year'].isin([2022, 2023])]

# Criando a tabela agregada
pivot = df_filtered.pivot_table(index='Month', columns='Year', values=['ValorTotal', 'UnidadesVendidas', 'Promo'], aggfunc='sum')

# Adicionando a linha com o valor total
pivot.loc['Total'] = pivot.sum()

pivot

Promo        UnidadesVendidas          ValorTotal             
Year    2022   2023             2022    2023        2022         2023
Month                                                                
1       39.0   17.0           1014.0   829.0   48038.830   42374.9645
2       31.0   28.0            725.0   744.0   33487.866   37752.1970
3       62.0   25.0           1455.0  1023.0   67337.112   54805.5775
4       37.0   15.0           1034.0   886.0   48180.612   45134.3005
5       45.0   20.0           1077.0   914.0   51761.596   46058.3785
6       26.0   16.0            715.0  1213.0   33658.754   64254.3245
7       48.0   23.0           1044.0  1079.0   48442.970   58027.6275
8       59.0   23.0           1126.0   968.0   50768.534   53043.2705
9       35.0   28.0            848.0  1246.0   40119.278   65966.1620
Total  382.0  195.0           9038.0  8902.0  421795.552  467416.8025

In [ ]:
# Extraindo a linha 'Total'
total_values = pivot.loc['Total']

# Transpondo a série para um DataFrame e resetando o índice
total_df = total_values.transpose().reset_index()

# Renomeando as colunas
total_df.columns = ['Category', 'Year', 'Value']

# Pivotando o DataFrame
pivot_total = total_df.pivot(index='Category', columns='Year', values='Value')

# Calculando a mudança percentual
pivot_total['Change (%)'] = (pivot_total[2023] - pivot_total[2022]) / pivot_total[2022] * 100

pivot_total

In [ ]:
# o ano de 2023 teve menos promoções 
# e menos Unidades vendidas em relação a 2022, 
# porém obteve um Valor Total maior 
# indicando que houve um Ticket Médio maior em 2023 em relação a 2022

In [ ]:
# Calculando o Ticket Médio para cada ano
pivot_total['Ticket Médio 2022'] = pivot_total[2022]['ValorTotal'] / pivot_total[2022]['UnidadesVendidas']
pivot_total['Ticket Médio 2023'] = pivot_total[2023]['ValorTotal'] / pivot_total[2023]['UnidadesVendidas']

# Exibindo o DataFrame
pivot_total

In [ ]:
import matplotlib.pyplot as plt

# Definindo o tamanho da figura
fig, ax = plt.subplots(figsize=(15, 10))  # Aumentando a largura da figura

# Adicionando a tabela à figura
table_data = pivot_total.reset_index().round(2).values.tolist()  # Arredondando os valores para 2 casas decimais
column_labels = pivot_total.reset_index().columns.tolist()

# Definindo as cores
colors = sns.color_palette("Pastel2").as_hex()

# Criando uma matriz de cores
cell_colors = [[colors[i%len(colors)] for _ in row] for i, row in enumerate(table_data)]

# Adicionando a tabela à figura
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc = 'center', loc='center', cellColours=cell_colors)

# Ajustando o tamanho da fonte
table.auto_set_font_size(False)
table.set_fontsize(14)

# Ajustando o tamanho da fonte dos cabeçalhos
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(10)

# Ajustando a largura das colunas e a altura das linhas
table.scale(1.2, 1.5)  # Aumentando a largura das colunas e a altura das linhas

# Escondendo os eixos
ax.axis('off')

# Salvando a figura como .png
plt.savefig('pivot_total_table.png')

In [ ]:
# Calculate 'Ticket Médio' for 2022 and 2023
pivot[('Ticket Médio', 2022)] = pivot[('ValorTotal', 2022)] / pivot[('UnidadesVendidas', 2022)]
pivot[('Ticket Médio', 2023)] = pivot[('ValorTotal', 2023)] / pivot[('UnidadesVendidas', 2023)]

# Display the updated DataFrame
pivot

In [ ]:

# Definindo o tamanho da figura
fig, ax = plt.subplots(figsize=(15, 10))  # Aumentando a largura da figura

# Adicionando a tabela à figura
table_data = pivot.reset_index().round(2).values.tolist()  # Arredondando os valores para 2 casas decimais
column_labels = pivot.reset_index().columns.tolist()

# Definindo as cores
colors = sns.color_palette("Pastel2").as_hex()

# Criando uma matriz de cores
cell_colors = [[colors[i%len(colors)] for _ in row] for i, row in enumerate(table_data)]

# Adicionando a tabela à figura
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc = 'center', loc='center', cellColours=cell_colors)

# Ajustando o tamanho da fonte
table.auto_set_font_size(False)
table.set_fontsize(16)

# Ajustando o tamanho da fonte dos cabeçalhos
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(13)

# Ajustando a largura das colunas e a altura das linhas
table.scale(2, 1.8)  # Aumentando a largura das colunas e a altura das linhas

# Escondendo os eixos
ax.axis('off')

# Salvando a figura como .png
plt.savefig('pivot_table_with_ticket_medio.png')

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define a list of colors
colors = ['#b3e2cd', '#fdcdac', '#cbd5e8', '#f4cae4', '#e6f5c9', '#fff2ae', '#f1e2cc', '#cccccc']

# Calcular o Ticket Médio
df_t['Ticket Médio'] = df_t['ValorTotal'] / df_t['UnidadesVendidas']

# Agrupar os dados e calcular a média
df_grouped = df_t.groupby(['Year', 'Month']).agg({'UnidadesVendidas': 'sum', 'ValorTotal': 'sum', 'Promo': 'sum', 'Ticket Médio': 'mean'}).reset_index()

# Obter o número de colunas
num_cols = len(df_grouped.columns) - 2  # Subtrair as colunas 'Year' e 'Month'

# Calcular o número de linhas e colunas para os subplots
num_rows = num_cols // 2 if num_cols % 2 == 0 else num_cols // 2 + 1

# Criar subplots
fig = make_subplots(rows=num_rows, cols=2, subplot_titles=df_grouped.columns[2:])

# Adicionar traços
for i, header in enumerate(df_grouped.columns[2:]):
    row = i // 2 + 1
    col = i % 2 + 1
    for j, year in enumerate(df_grouped['Year'].unique()):
        df_year = df_grouped[df_grouped['Year'] == year]
        fig.add_trace(go.Scatter(x=df_year['Month'], y=df_year[header], mode='lines', name=f'{header} {year}', line=dict(color=colors[j % len(colors)])), row=row, col=col)

# Atualizar layout
fig.update_layout(height=600, width=800, title_text="Subplots")
fig.show()

#### Segunda parte da análise:
- explorando os dados de transações com as Marcas e suas categorias

##### nesta segunda parte da análise é possível verificar que:

###### O top vermelho gammaz - teve ótimo desempenho,
###### - ficou em segundo lugar no rank de Ticket Médio (TM) no ano de 2023. Caiu 1 posição em relação a 2022
###### - se manteve em primeiro em relação a Unidades Vendidas e Valor Total (VT) 2022 e 2023
###### - fez menos promoções em 2023 em relação a 2022 - caiu de sexto em quantidade de promoções para sétimo
###### mas o Botton azul Gammaz também teve destaque!

###### 2023	Bottom Azul Marca GammaZ  ->	TM - 88.517723	 VT - 75759.52
###### 2023	Top Vermelho Marca GammaZ ->	TM - 83.774699	 VT - 146329.22

###### O Botton azul gammaz que não foi vendido em 2022 teve um desempenho melhor em 2023 quando se olha pra Ticket Médio.

###### A Marca Gammaz no geral teve um bom desempenho pra Ticket Médio e Valor Total, ultrapassando em Valor Total a Marca AlfaX
 
###### As promoções não impulsionaram o negócio de forma geral

###### Cores vermelhas são representativas em vendas, mas um destaque foram peças azuis, que melhoraram tanto em quantidade vendida, quanto em aumento de ticket médio e valor Total

###### Houve crescimento de Ticket médio de peças tipo Botton e também Valor Total, o que ajuda a explicar o sucesso do Botton azul gammaz


In [ ]:
# O top vermelho gammaz - teve ótimo desempenho, mas o Botton azul Gammaz também teve destaque
# - ficou em segundo lugar no rank de Ticket Médio no ano de 2023. Caiu 1 posição em relação a 2022
# - se manteve em primeiro em relação a Unidades Vendidas e Valor Total 2022 e 2023
# - fez menos promoções em 2023 em relação a 2022 - caiu de sexto em quantidade de promoções para sétimo

# 2023	Bottom Azul Marca GammaZ	TM - 88.517723	VT - 75759.52
# 2023	Top Vermelho Marca GammaZ	TM - 83.774699	VT - 146329.22

# O Botton azul gammaz que não foi vendido em 2022 teve um desempenho melhor em 2023 quando se olha pra Ticket Médio.

# A Marca Gammaz no geral teve um bom desempenho pra Ticket Médio e Valor Total, ultrapassando em Valor Total a Marca AlfaX
 
# As promoções não impulsionaram o negócio de forma geral

# Cores vermelhas são representativas em vendas, mas um destaque foram peças azuis que melhoraram tanto em quantidade vendida quanto em aumento de ticket médio e valor Total

# Houve crescimento de Ticket médio de peças tipo Botton e também Valor Total, o que ajuda a explicar o sucesso do Botton azul gammaz

In [ ]:
import openpyxl
import pandas as pd

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de produto
ws = wb['Produto']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_p = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_p.columns:
    # Se o tipo de dados da coluna é object (string)
    if df_p[col].dtype == 'object':
        # Remova os espaços em branco no início e no final  
        df_p[col] = df_p[col].str.strip()

In [ ]:
df_p.info()

In [ ]:
# verificando valores nulos
null_counts = df_p.isnull().sum()
print(null_counts)


In [ ]:
# verificando valores duplicados

for column in df_p.columns:
    print(f"\nColumn: {column}")
    print(df_p[column].value_counts(dropna=False))


In [ ]:
df_p

In [ ]:
def update_description(row):
    if row['Categoria'] not in row['Descrição Produto']:
        row['Descrição Produto'] = row['Categoria'] + ' ' + row['Descrição Produto']
    if row['Cor'] not in row['Descrição Produto']:
        row['Descrição Produto'] = row['Descrição Produto'].replace(row['Categoria'], row['Categoria'] + ' ' + row['Cor'])
    if 'Marca' not in row['Descrição Produto'] and row['Marca'] not in row['Descrição Produto']:
        row['Descrição Produto'] = row['Descrição Produto'].replace(row['Marca'], 'Marca ' + row['Marca'])
    return row

df_p = df_p.apply(update_description, axis=1)

df_p.loc[df_p['ID_Produto'] == 123010, 'Descrição Produto'] = 'Bottom Azul Marca GammaZ'
df_p.loc[df_p['ID_Produto'] == 123011, 'Descrição Produto'] = 'Bottom Azul Marca BetaY'

In [ ]:
df_p

In [ ]:
df_t['ID_Produto'].value_counts(dropna=False)

In [ ]:
df_p['ID_Produto'].value_counts(dropna=False)

In [ ]:
#left join
df_tp = pd.merge(df_t, df_p, on='ID_Produto', how='left')

In [ ]:
df_tp.head()

In [ ]:
# liberando memória
del df_t

In [ ]:
df_tp.info()

In [ ]:
# Drop unnecessary columns
df_tp_drp = df_tp.drop(columns=['ID_Trans', 'Data', 'ID_Loja', 'ID_Produto', 'ID_Cliente', 'Month'])

# Group by 'Descrição Produto', 'Categoria', 'Cor', 'Marca', and 'Year' and calculate the mean of 'Ticket Médio' and the sum of 'UnidadesVendidas', 'ValorTotal', and 'Promo'
grouped_df = df_tp_drp.groupby(['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Year']).agg({'Ticket Médio': 'mean', 'UnidadesVendidas': 'sum', 'ValorTotal': 'sum', 'Promo': 'sum'}).reset_index()

grouped_df

In [ ]:
grouped_df.info()

In [ ]:
# Filter the DataFrame by year and create two separate DataFrames
df1 = grouped_df[grouped_df['Year'] == 2022].copy()
df2 = grouped_df[grouped_df['Year'] == 2023].copy()

# Create a rank of 'UnidadesVendidas' in each DataFrame
df1['Rank_2022_UnidadesVendidas'] = df1['UnidadesVendidas'].rank(ascending=False)
df2['Rank_2023_UnidadesVendidas'] = df2['UnidadesVendidas'].rank(ascending=False)

# Merge the two DataFrames using a full outer join, bringing only the rank columns
rank_UnidadesVendidas = pd.merge(df1[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2022_UnidadesVendidas']], 
                                 df2[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2023_UnidadesVendidas']], 
                                 on=['Descrição Produto', 'Categoria', 'Cor', 'Marca'], 
                                 how='outer')

# Sort the DataFrame by 'Rank_2023_UnidadesVendidas'
rank_UnidadesVendidas = rank_UnidadesVendidas.sort_values('Rank_2023_UnidadesVendidas')

rank_UnidadesVendidas

In [ ]:
# Filter the DataFrame by year and create two separate DataFrames
df1 = grouped_df[grouped_df['Year'] == 2022].copy()
df2 = grouped_df[grouped_df['Year'] == 2023].copy()

# Create a rank of 'ValorTotal' in each DataFrame
df1['Rank_2022_ValorTotal'] = df1['ValorTotal'].rank(ascending=False)
df2['Rank_2023_ValorTotal'] = df2['ValorTotal'].rank(ascending=False)

# Merge the two DataFrames using a full outer join, bringing only the rank columns
rank_ValorTotal = pd.merge(df1[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2022_ValorTotal']], 
                           df2[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2023_ValorTotal']], 
                           on=['Descrição Produto', 'Categoria', 'Cor', 'Marca'], 
                           how='outer')

# Sort the DataFrame by 'Rank_2023_ValorTotal'
rank_ValorTotal = rank_ValorTotal.sort_values('Rank_2023_ValorTotal')

rank_ValorTotal

In [ ]:
# Filter the DataFrame by year and create two separate DataFrames
df1 = grouped_df[grouped_df['Year'] == 2022].copy()
df2 = grouped_df[grouped_df['Year'] == 2023].copy()

# Create a rank of 'Promo' in each DataFrame
df1['Rank_2022_Promo'] = df1['Promo'].rank(ascending=False)
df2['Rank_2023_Promo'] = df2['Promo'].rank(ascending=False)

# Merge the two DataFrames using a full outer join, bringing only the rank columns
rank_Promo = pd.merge(df1[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2022_Promo']], 
                      df2[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2023_Promo']], 
                      on=['Descrição Produto', 'Categoria', 'Cor', 'Marca'], 
                      how='outer')

# Sort the DataFrame by 'Rank_2023_Promo'
rank_Promo = rank_Promo.sort_values('Rank_2023_Promo')

rank_Promo

In [ ]:
# Filter the DataFrame by year and create two separate DataFrames
df1 = grouped_df[grouped_df['Year'] == 2022].copy()
df2 = grouped_df[grouped_df['Year'] == 2023].copy()

# Create a rank of 'Ticket Médio' in each DataFrame
df1['Rank_2022_Ticket_Medio'] = df1['Ticket Médio'].rank(ascending=False)
df2['Rank_2023_Ticket_Medio'] = df2['Ticket Médio'].rank(ascending=False)

# Merge the two DataFrames using a full outer join, bringing only the rank columns
rank_Ticket_Medio = pd.merge(df1[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2022_Ticket_Medio']], 
                             df2[['Descrição Produto', 'Categoria', 'Cor', 'Marca', 'Rank_2023_Ticket_Medio']], 
                             on=['Descrição Produto', 'Categoria', 'Cor', 'Marca'], 
                             how='outer')

# Sort the DataFrame by 'Rank_2023_Ticket_Medio'
rank_Ticket_Medio = rank_Ticket_Medio.sort_values('Rank_2023_Ticket_Medio')

rank_Ticket_Medio

In [ ]:
# Drop 'Categoria' and 'Marca' columns
rank_UnidadesVendidas = rank_UnidadesVendidas.drop(['Categoria', 'Marca','Cor'], axis=1)
rank_ValorTotal = rank_ValorTotal.drop(['Categoria', 'Marca','Cor'], axis=1)
rank_Promo = rank_Promo.drop(['Categoria', 'Marca','Cor'], axis=1)
rank_Ticket_Medio = rank_Ticket_Medio.drop(['Categoria', 'Marca','Cor'], axis=1)

# Merge all DataFrames
merged_df = rank_UnidadesVendidas.merge(rank_ValorTotal, how='outer', on=['Descrição Produto'])
merged_df = merged_df.merge(rank_Promo, how='outer', on=['Descrição Produto'])
merged_df = merged_df.merge(rank_Ticket_Medio, how='outer', on=['Descrição Produto'])

# Sort by 'Rank_2023_Ticket_Medio'
merged_df = merged_df.sort_values(by='Rank_2023_Ticket_Medio')

In [ ]:
cols = ['Descrição Produto', 'Rank_2022_Ticket_Medio', 'Rank_2023_Ticket_Medio'] + [col for col in merged_df.columns if col not in ['Descrição Produto', 'Rank_2022_Ticket_Medio', 'Rank_2023_Ticket_Medio']]
merged_df = merged_df[cols]


In [ ]:
merged_df.head(4)

In [ ]:
# Define a dictionary for the new column names
new_column_names = {
    'Rank_2022_Ticket_Medio': 'Rank_22_TM',
    'Rank_2023_Ticket_Medio': 'Rank_23_TM',
    'Rank_2022_UnidadesVendidas': 'Rank_22_UV',
    'Rank_2023_UnidadesVendidas': 'Rank_23_UV',
    'Rank_2022_ValorTotal': 'Rank_22_VT',
    'Rank_2023_ValorTotal': 'Rank_23_VT',
    'Rank_2022_Promo': 'Rank_22_Pro',
    'Rank_2023_Promo': 'Rank_23_Pro',
    'Descrição Produto': 'DescProduto'
}

# Rename the columns
merged_df.rename(columns=new_column_names, inplace=True)

In [ ]:
merged_df.info()

In [ ]:
# Defining the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Adding the table to the figure
table_data = merged_df.reset_index().round(2).values.tolist()
column_labels = merged_df.reset_index().columns.tolist()

# Defining the colors
colors = sns.color_palette("Pastel2").as_hex()

# Creating a color matrix
cell_colors = [[colors[i%len(colors)] for _ in row] for i, row in enumerate(table_data)]

# Adding the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc = 'center', loc='center', cellColours=cell_colors)

# Adjusting the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(4, 3.5)  # Increase the first value to increase the width, and the second value to increase the height

# Increase the font size
table.set_fontsize(20)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(15)

# Adjust the font size of 'DescProduto' column
for key, cell in table.get_celld().items():
    if key[1] == 0:  # 0 is the index of 'DescProduto' column
        cell.set_fontsize(14)  # Increase this value to make the text larger
# Hiding the axes
ax.axis('off')

# Saving the figure as .png
plt.savefig('merged_df_table.png')

In [ ]:
# Create DataFrame with columns that contain '2022'
df_2022 = merged_df.filter(regex='DesProduto|2022')

# Create DataFrame with columns that contain '2023'
df_2023 = merged_df.filter(regex='DesProduto|2023')

In [ ]:
import plotly.figure_factory as ff

# Create DataFrame with columns that contain '22'
df_2022 = merged_df.filter(regex='22')

# Create DataFrame with columns that contain '23'
df_2023 = merged_df.filter(regex='23')

# Create correlation matrices and round to 3 decimal places
correlation_matrix_2022 = df_2022.corr().round(3)
correlation_matrix_2023 = df_2023.corr().round(3)

# Create heatmaps with 'BluGrn' color scale and smaller font size
heatmap_2022 = ff.create_annotated_heatmap(z=correlation_matrix_2022.values, 
                                           x=list(correlation_matrix_2022.columns), 
                                           y=list(correlation_matrix_2022.index), 
                                           colorscale='BluGrn', 
                                           font_colors=['black', 'white'], 
                                           annotation_text=correlation_matrix_2022.values.astype(str))

heatmap_2023 = ff.create_annotated_heatmap(z=correlation_matrix_2023.values, 
                                           x=list(correlation_matrix_2023.columns), 
                                           y=list(correlation_matrix_2023.index), 
                                           colorscale='BluGrn', 
                                           font_colors=['black', 'white'], 
                                           annotation_text=correlation_matrix_2023.values.astype(str))

# Add titles
heatmap_2022.update_layout(title='Heatmap 2022')
heatmap_2023.update_layout(title='Heatmap 2023')

# Decrease the size of the axis labels
heatmap_2022.update_xaxes(tickfont=dict(size=10))
heatmap_2022.update_yaxes(tickfont=dict(size=10))
heatmap_2023.update_xaxes(tickfont=dict(size=10))
heatmap_2023.update_yaxes(tickfont=dict(size=10))

# Display heatmaps
heatmap_2022.show()
heatmap_2023.show()

In [ ]:
# Group by 'Year' and 'Marca' and calculate the sum of 'ValorTotal' and the mean of 'Ticket Médio'
df_Marca = df_tp.groupby(['Year', 'Marca']).agg({'ValorTotal': 'sum', 'Ticket Médio': 'mean'}).reset_index()

# Display the new DataFrame
print(df_Marca)

In [ ]:
# Group by 'Year' and 'Marca' and calculate the sum of 'ValorTotal' and the mean of 'Ticket Médio'
df_Marca_ = df_tp.groupby(['Year', 'Marca']).agg({'Promo': 'sum', 'UnidadesVendidas': 'sum'}).reset_index()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Marca['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Marca[df_Marca['Year'] == year]

    # Create a bar plot for 'Ticket Médio'
    fig.add_trace(go.Bar(x=df_year['Marca'], y=df_year['Ticket Médio'], name='Ticket Médio ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'ValorTotal'
    fig.add_trace(go.Bar(x=df_year['Marca'], y=df_year['ValorTotal'], name='ValorTotal ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Marca", row=1, col=1)
fig.update_xaxes(title_text="Marca", row=1, col=2)
fig.update_yaxes(title_text="Ticket Médio", row=1, col=1)
fig.update_yaxes(title_text="ValorTotal", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Marca_['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Marca_[df_Marca_['Year'] == year]

    # Create a bar plot for 'Promo'
    fig.add_trace(go.Bar(x=df_year['Marca'], y=df_year['Promo'], name='Promo ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'UnidadesVendidas'
    fig.add_trace(go.Bar(x=df_year['Marca'], y=df_year['UnidadesVendidas'], name='UnidadesVendidas ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Marca", row=1, col=1)
fig.update_xaxes(title_text="Marca", row=1, col=2)
fig.update_yaxes(title_text="Promo", row=1, col=1)
fig.update_yaxes(title_text="UnidadesVendidas", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
# GammaZ tem o maior Ticket Médio em 2022 e 2023 e ultrapasou a marca AlphaX em 2023 em relação ao Valor Total
# As promoções não teem efeito significativo nas vendas de UnidadesVendidas para nenhuma das marcas em 2022 e 2023

In [ ]:
# Group by 'Year' and 'Cor' and calculate the sum of 'ValorTotal' and the mean of 'Ticket Médio'
df_Cor = df_tp.groupby(['Year', 'Cor']).agg({'ValorTotal': 'sum', 'Ticket Médio': 'mean'}).reset_index()

# Group by 'Year' and 'Cor' and calculate the sum of 'Promo' and 'UnidadesVendidas'
df_Cor_ = df_tp.groupby(['Year', 'Cor']).agg({'Promo': 'sum', 'UnidadesVendidas': 'sum'}).reset_index()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Cor['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Cor[df_Cor['Year'] == year]

    # Create a bar plot for 'Ticket Médio'
    fig.add_trace(go.Bar(x=df_year['Cor'], y=df_year['Ticket Médio'], name='Ticket Médio ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'ValorTotal'
    fig.add_trace(go.Bar(x=df_year['Cor'], y=df_year['ValorTotal'], name='ValorTotal ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Cor", row=1, col=1)
fig.update_xaxes(title_text="Cor", row=1, col=2)
fig.update_yaxes(title_text="Ticket Médio", row=1, col=1)
fig.update_yaxes(title_text="ValorTotal", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Cor_['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Cor_[df_Cor_['Year'] == year]

    # Create a bar plot for 'Promo'
    fig.add_trace(go.Bar(x=df_year['Cor'], y=df_year['Promo'], name='Promo ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'UnidadesVendidas'
    fig.add_trace(go.Bar(x=df_year['Cor'], y=df_year['UnidadesVendidas'], name='UnidadesVendidas ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Cor", row=1, col=1)
fig.update_xaxes(title_text="Cor", row=1, col=2)
fig.update_yaxes(title_text="Promo", row=1, col=1)
fig.update_yaxes(title_text="UnidadesVendidas", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
# apenas a cor azul respondeu positivamente a promoções em 2023 em relação a unidades vendidas

In [ ]:
# Group by 'Year' and 'Categoria' and calculate the sum of 'ValorTotal' and the mean of 'Ticket Médio'
df_Categoria = df_tp.groupby(['Year', 'Categoria']).agg({'ValorTotal': 'sum', 'Ticket Médio': 'mean'}).reset_index()

# Group by 'Year' and 'Categoria' and calculate the sum of 'Promo' and 'UnidadesVendidas'
df_Categoria_ = df_tp.groupby(['Year', 'Categoria']).agg({'Promo': 'sum', 'UnidadesVendidas': 'sum'}).reset_index()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Categoria['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Categoria[df_Categoria['Year'] == year]

    # Create a bar plot for 'Ticket Médio'
    fig.add_trace(go.Bar(x=df_year['Categoria'], y=df_year['Ticket Médio'], name='Ticket Médio ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'ValorTotal'
    fig.add_trace(go.Bar(x=df_year['Categoria'], y=df_year['ValorTotal'], name='ValorTotal ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Categoria", row=1, col=1)
fig.update_xaxes(title_text="Categoria", row=1, col=2)
fig.update_yaxes(title_text="Ticket Médio", row=1, col=1)
fig.update_yaxes(title_text="ValorTotal", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Categoria_['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Categoria_[df_Categoria_['Year'] == year]

    # Create a bar plot for 'Promo'
    fig.add_trace(go.Bar(x=df_year['Categoria'], y=df_year['Promo'], name='Promo ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'UnidadesVendidas'
    fig.add_trace(go.Bar(x=df_year['Categoria'], y=df_year['UnidadesVendidas'], name='UnidadesVendidas ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Categoria", row=1, col=1)
fig.update_xaxes(title_text="Categoria", row=1, col=2)
fig.update_yaxes(title_text="Promo", row=1, col=1)
fig.update_yaxes(title_text="UnidadesVendidas", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
# Group by 'Year' and 'Descrição Produto' and calculate the sum of 'ValorTotal' and the mean of 'Ticket Médio'
df_Descrição_Produto = df_tp.groupby(['Year', 'Descrição Produto']).agg({'ValorTotal': 'sum', 'Ticket Médio': 'mean'}).reset_index()

# Group by 'Year' and 'Descrição Produto' and calculate the sum of 'Promo' and 'UnidadesVendidas'
df_Descrição_Produto_ = df_tp.groupby(['Year', 'Descrição Produto']).agg({'Promo': 'sum', 'UnidadesVendidas': 'sum'}).reset_index()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Descrição_Produto['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Descrição_Produto[df_Descrição_Produto['Year'] == year]

    # Create a bar plot for 'Ticket Médio'
    fig.add_trace(go.Bar(x=df_year['Descrição Produto'], y=df_year['Ticket Médio'], name='Ticket Médio ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'ValorTotal'
    fig.add_trace(go.Bar(x=df_year['Descrição Produto'], y=df_year['ValorTotal'], name='ValorTotal ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Descrição Produto", row=1, col=1)
fig.update_xaxes(title_text="Descrição Produto", row=1, col=2)
fig.update_yaxes(title_text="Ticket Médio", row=1, col=1)
fig.update_yaxes(title_text="ValorTotal", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 1 row and 2 columns, with titles for spacing
fig = make_subplots(rows=1, cols=2, subplot_titles=("",""))

# Define colors for each year
colors = {'2022': ['rgb(179,226,205)', 'rgb(253,205,172)'], '2023': ['rgb(102,194,165)', 'rgb(252,141,98)']}

# Get the unique years
years = df_Descrição_Produto_['Year'].unique()

# For each year
for i, year in enumerate(years):
    # Filter the DataFrame for the current year
    df_year = df_Descrição_Produto_[df_Descrição_Produto_['Year'] == year]

    # Create a bar plot for 'Promo'
    fig.add_trace(go.Bar(x=df_year['Descrição Produto'], y=df_year['Promo'], name='Promo ' + str(year), marker_color=colors[str(year)][0]), row=1, col=1)

    # Create a bar plot for 'UnidadesVendidas'
    fig.add_trace(go.Bar(x=df_year['Descrição Produto'], y=df_year['UnidadesVendidas'], name='UnidadesVendidas ' + str(year), marker_color=colors[str(year)][1]), row=1, col=2)

# Update xaxis and yaxis properties
fig.update_xaxes(title_text="Descrição Produto", row=1, col=1)
fig.update_xaxes(title_text="Descrição Produto", row=1, col=2)
fig.update_yaxes(title_text="Promo", row=1, col=1)
fig.update_yaxes(title_text="UnidadesVendidas", row=1, col=2)

# Update layout to group bars instead of stacking, increase figure width, and adjust subplot spacing
fig.update_layout(barmode='group', width=1200, margin=dict(b=100))

# Show the figure
fig.show()

In [ ]:
import itertools 
# Get unique products and years
products = df_tp['Descrição Produto'].unique()
years = df_tp['Year'].unique()

# Create subplots
fig, axs = plt.subplots(len(products)*len(years), 3, figsize=(15, 5*len(products)*len(years)))

for i, (product, year) in enumerate(itertools.product(products, years)):
    # Filter DataFrame by product and year
    df_product_year = df_tp[(df_tp['Descrição Produto'] == product) & (df_tp['Year'] == year)]

    # Create distribution plots
    sns.distplot(df_product_year['Ticket Médio'], ax=axs[i, 0])
    sns.distplot(df_product_year['UnidadesVendidas'], ax=axs[i, 1])
    sns.distplot(df_product_year['ValorTotal'], ax=axs[i, 2])

    # Set titles
    axs[i, 0].set_title(f'Ticket Médio - {product} - {year}')
    axs[i, 1].set_title(f'UnidadesVendidas - {product} - {year}')
    axs[i, 2].set_title(f'ValorTotal - {product} - {year}')

plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

def create_plot_for_brand(brand):
    # Get unique products and years for the brand
    products = df_tp[df_tp['Marca'] == brand]['Descrição Produto'].unique()
    years = sorted(df_tp[df_tp['Marca'] == brand]['Year'].unique())

    # Define color sequence
    colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']

    # Create a dictionary to map years to colors
    year_color = {year: colors[i % len(colors)] for i, year in enumerate(years)}

    # Create dictionaries to store mean 'Ticket Médio' and sum 'UnidadesVendidas', 'ValorTotal' and 'Promo' for each product and year
    ticket_medio_means = {year: {} for year in years}
    unidades_vendidas_sums = {year: {} for year in years}
    valor_total_sums = {year: {} for year in years}
    promo_sums = {year: {} for year in years}

    fig = go.Figure()
    for year in years:
        for product in products:
            # Filter DataFrame by product and year
            df_product_year = df_tp[(df_tp['Marca'] == brand) & (df_tp['Descrição Produto'] == product) & (df_tp['Year'] == year)]

            # Calculate mean 'Ticket Médio' and sum 'UnidadesVendidas', 'ValorTotal' and 'Promo'
            ticket_medio_mean = df_product_year['Ticket Médio'].mean()
            unidades_vendidas_sum = df_product_year['UnidadesVendidas'].sum()
            valor_total_sum = df_product_year['ValorTotal'].sum()
            promo_sum = df_product_year['Promo'].sum()

            # Store these values in the dictionaries
            ticket_medio_means[year][product] = ticket_medio_mean
            unidades_vendidas_sums[year][product] = unidades_vendidas_sum
            valor_total_sums[year][product] = valor_total_sum
            promo_sums[year][product] = promo_sum

        # Add a bar or line to the chart depending on the year
        if year == 2022:
            fig.add_trace(go.Bar(x=products, y=[ticket_medio_means[year][product] for product in products], name=f'{year}', marker_color=year_color[year],
                                 hovertemplate='Ticket Médio: %{y:.2f}<br>Unidades Vendidas: %{customdata[0]:.2f}<br>Valor Total: %{customdata[1]:.2f}<br>Promo: %{customdata[2]:.2f}<extra></extra>',
                                 customdata=[[unidades_vendidas_sums[year][product], valor_total_sums[year][product], promo_sums[year][product]] for product in products]))
        elif year == 2023:
            # Calculate percentage variation from 2022 to 2023 for each product
            percent_variations = [(ticket_medio_means[year][product] / ticket_medio_means[2022][product] - 1) * 100 for product in products]

            fig.add_trace(go.Scatter(x=products, y=[ticket_medio_means[year][product] for product in products], mode='lines', name=f'{year}', line=dict(color=year_color[year]),
                                     hovertemplate='Ticket Médio: %{y:.2f}<br>Variation from 2022: %{customdata[0]:.2f}%<br>Unidades Vendidas: %{customdata[1]:.2f}<br>Valor Total: %{customdata[2]:.2f}<br>Promo: %{customdata[3]:.2f}<extra></extra>',
                                     customdata=[[percent_variations[i], unidades_vendidas_sums[year][product], valor_total_sums[year][product], promo_sums[year][product]] for i, product in enumerate(products)]))

    # Update layout
    fig.update_layout(
        height=600, 
        width=900, 
        title_text=f"Ticket Médio for {brand}", 
        xaxis_title="Descrição Produto", 
        yaxis_title="Ticket Médio", 
        barmode='group',
        xaxis_tickangle=-90
    )

    # Show plot
    fig.show()

# Get unique brands
brands = df_tp['Marca'].unique()

# Create a plot for each brand
for brand in brands:
    create_plot_for_brand(brand)

In [ ]:
# grouped = df_sorted.groupby('Year')

# for name, group in grouped:
#     print(f"Year: {name}")
#     display(group)

In [ ]:
pivot_df = df_sorted.pivot_table(index=['Marca', 'Descrição Produto'], columns='Year', values=['Ticket Médio', 'ValorTotal', 'UnidadesVendidas'])

for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    pivot_df[(col, 'Variação 2022-2023')] = ((pivot_df[(col, 2023)] - pivot_df[(col, 2022)]) / pivot_df[(col, 2022)]) * 100

pivot_df = pivot_df.fillna(0)

pivot_df = pivot_df.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)


# Defining the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Adding the table to the figure
table_data = pivot_df.reset_index().round(2).values.tolist()
column_labels = pivot_df.reset_index().columns.tolist()

# Create a color dictionary for each 'Marca'
marca_colors = {marca: colors[i % len(colors)] for i, marca in enumerate(pivot_df.index.get_level_values('Marca').unique())}

# Create a color matrix based on 'Marca'
cell_colors = [[marca_colors[row[0]] for _ in row] for row in table_data]


# Adding the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc = 'center', loc='center', cellColours=cell_colors)

# Adjusting the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)  # Increase these values to increase the width and height of the columns

# Increase the font size
table.set_fontsize(25)  # Increase this value to make the text larger

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)  # Increase this value to make the text larger

# Adjust the font size of 'Marca' and 'Descrição Produto' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:  # 0 and 1 are the indices of 'Marca' and 'Descrição Produto' columns
        cell.set_fontsize(18)  # Increase this value to make the text larger

# Hiding the axes
ax.axis('off')

# Saving the figure as .png
plt.savefig('pivot_df_table.png')

In [ ]:
# liberando espaço na memória
# Get a dictionary of all variables in your current namespace
variables = locals()

# Convert the items to a list before iterating
for var_name, var_value in list(variables.items()):
    if isinstance(var_value, pd.DataFrame) and var_name != 'df_tp':
        del variables[var_name]

##### Quarta parte da análise:
- Clientes e Lojas

###### nesta quarta parte da análise é possível verificar que:

- As lojas 5(SP), 8(RS) e 2(MG) foram as que melhor performaram em relação a Ticket Médio no ano de 2023
- Para estas lojas a Classe B é a com maior Ticket Médio no ano de 2023 e o sexo feminino se destaca neste quesito
- nas faixas etárias há um diferença entre as lojas quando se trata de Ticket Médio no ano de 2023
loja 5 -> 18-25
loja 8 -> 25-35
loja 2 -> 45+
- as tabelas trazem detalhes que podem auxiliar em decisões futuras de MKT de froma mais detalhada

In [ ]:
import openpyxl
import pandas as pd

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de Cliente
ws = wb['Cliente']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_c = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_c.columns:
    # Se o tipo de dados da coluna é object (string)
    if pd.api.types.is_object_dtype(df_c[col]):
        # Remova os espaços em branco no início e no final
        df_c[col] = df_c[col].str.strip()

In [ ]:
import openpyxl
import pandas as pd

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de Cliente
ws = wb['Lojas']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_l = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_l.columns:
    # Se o tipo de dados da coluna é object (string)
    if pd.api.types.is_object_dtype(df_l[col]):
        # Remova os espaços em branco no início e no final
        df_l[col] = df_l[col].str.strip()

In [ ]:
df_l.info()

In [ ]:
df_l

In [ ]:
# If 'Regiao' is None and 'UF' is 'SP', set 'Regiao' to 'SUDESTE'
df_l.loc[(df_l['Regiao'].isnull()) & (df_l['UF'] == 'SP'), 'Regiao'] = 'SUDESTE'

# If 'Regiao' is None and 'UF' is 'BH', set 'Regiao' to 'SUDESTE'
df_l.loc[(df_l['Regiao'].isnull()) & (df_l['UF'] == 'BH'), 'Regiao'] = 'SUDESTE'

In [ ]:
df_l

In [ ]:
df_c.info()

In [ ]:
df_c = df_c.dropna(axis=1, how='all')
df_c[df_c['Genero'].isnull()]
for col in df_c.columns:
    print(f'Column: {col}')
    print(df_c[col].value_counts(dropna=False))

In [ ]:
df_c.info()

In [ ]:
df_tp.info()

In [ ]:
# Perform a left join of df_tp and df_l on the 'ID_Loja' column
df_merged = df_tp.merge(df_l, on='ID_Loja', how='left')

In [ ]:
df_merged.info()

In [ ]:
# Perform a left join of df_merged and df_c on the 'ID_Cliente' and 'CustomerID' columns
df_final = df_merged.merge(df_c, left_on='ID_Cliente', right_on='CustomerID', how='left')

In [ ]:
df_final.info()

In [ ]:
# liberando espaço na memória
# Get a dictionary of all variables in your current namespace
variables = locals()

# Convert the items to a list before iterating
for var_name, var_value in list(variables.items()):
    if isinstance(var_value, pd.DataFrame) and var_name != 'df_final':
        del variables[var_name]

In [ ]:
# Group by 'ID_Loja' and 'Year' and calculate the mean of 'Ticket Médio', 'ValorTotal', and 'UnidadesVendidas'
df_grouped = df_final.groupby(['ID_Loja', 'Year']).agg({'Ticket Médio': 'mean', 'ValorTotal': 'sum', 'UnidadesVendidas': 'sum'})

# Rank 'ID_Loja' by 'Ticket Médio', 'ValorTotal', and 'UnidadesVendidas' for each year
df_grouped['Rank_Ticket_Medio'] = df_grouped.groupby('Year')['Ticket Médio'].rank(ascending=False)
df_grouped['Rank_ValorTotal'] = df_grouped.groupby('Year')['ValorTotal'].rank(ascending=False)
df_grouped['Rank_UnidadesVendidas'] = df_grouped.groupby('Year')['UnidadesVendidas'].rank(ascending=False)


In [ ]:
# Sort df_grouped by 'ID_Loja', 'Year' and 'Rank_Ticket_Medio'
df_sorted = df_grouped.sort_values(by=['ID_Loja', 'Rank_Ticket_Medio', 'Year'])

# Print the sorted DataFrame
df_sorted

In [ ]:
import plotly.graph_objects as go

# Reset the index
df_sorted_reset = df_sorted.reset_index()

# Create a color sequence similar to 'Pastel2'
color_sequence = ['#b3e2cd', '#fdcdac', '#cbd5e8', '#f4cae4', '#e6f5c9', '#fff2ae']

# Get the 'ID_Loja' with the best rank in 2023
best_rank_id_loja_2023 = df_sorted_reset[(df_sorted_reset['Year'] == 2023) & (df_sorted_reset['Rank_Ticket_Medio'] == df_sorted_reset['Rank_Ticket_Medio'].min())]['ID_Loja'].values[0]

# Create a bar plot
fig = go.Figure()

for i, year in enumerate(df_sorted_reset['Year'].unique()):
    df_year = df_sorted_reset[df_sorted_reset['Year'] == year]
    fig.add_trace(go.Bar(
        x=df_year['ID_Loja'],
        y=df_year['Ticket Médio'],
        name=str(year),
        text=df_year['Rank_Ticket_Medio'],
        legendgroup=str(year),
        marker_color=[color_sequence[i % len(color_sequence)] if id_loja != best_rank_id_loja_2023 or year != 2023 else '#ff0000' for id_loja in df_year['ID_Loja']],  # Highlight the best rank in 2023
        hovertemplate='ID_Loja: %{x}<br>Ticket Médio: %{y}<br>Year: '+str(year)+'<br>Rank: %{text}<extra></extra>',
    ))

# Add title and set barmode to 'group'
fig.update_layout(title_text='Ticket Médio by ID_Loja and Year', barmode='group')

# Show the plot
fig.show()

In [ ]:
df_grouped_unique = df_final.groupby(['ID_Loja', 'Year','Marca']).agg({'Ticket Médio': 'mean',
    'UnidadesVendidas': 'sum',
    'ValorTotal': 'sum'})
df_pivot = df_grouped_unique.reset_index().pivot_table(index=['ID_Loja', 'Marca'], columns='Year', values=['Ticket Médio', 'UnidadesVendidas', 'ValorTotal'])


df_sorted_2023 = df_sorted.loc[(slice(None), 2023), ['Rank_Ticket_Medio']]
df_sorted_2023_sorted = df_sorted_2023.sort_values(by='Rank_Ticket_Medio')
id_loja_list = df_sorted_2023_sorted.index.get_level_values('ID_Loja').tolist()


df_pivot_reindexed = df_pivot.reindex(id_loja_list, level='ID_Loja')
df_pivot_reindexed

In [ ]:
df_pivot_reindexed.info()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate the variation for each column
for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    df_pivot_reindexed[(col, 'Variação 2022-2023')] = ((df_pivot_reindexed[(col, 2023)] - df_pivot_reindexed[(col, 2022)]) / df_pivot_reindexed[(col, 2022)]) * 100

# Fill NaN values with 0
df_pivot_reindexed = df_pivot_reindexed.fillna(0)

# Reindex the columns
df_pivot_reindexed = df_pivot_reindexed.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)

# Define the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Add the table to the figure
table_data = df_pivot_reindexed.reset_index().round(2).values.tolist()
column_labels = df_pivot_reindexed.reset_index().columns.tolist()

# Create a color dictionary for each 'ID_Loja'
id_loja_colors = {id_loja: colors[i % len(colors)] for i, id_loja in enumerate(df_pivot_reindexed.index.get_level_values('ID_Loja').unique())}

# Create a color matrix based on 'ID_Loja'
cell_colors = [[id_loja_colors[row[0]] for _ in row] for row in table_data]

# Highlight the largest numbers in each column, skipping the first two columns
for i in range(2, len(table_data[0])):
    column_data = [row[i] for row in table_data]
    max_value = max(column_data)
    for j in range(len(table_data)):
        if table_data[j][i] == max_value:
            cell_colors[j][i] = 'red'  # Change 'red' to any color you want

# Add the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center', cellColours=cell_colors)

# Adjust the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)

# Increase the font size
table.set_fontsize(25)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)

# Adjust the font size of 'ID_Loja' and 'Marca' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:
        cell.set_fontsize(18)

# Hide the axes
ax.axis('off')

# Save the figure as .png
plt.savefig('df_pivot_reindexed_table.png')

In [ ]:
df_grouped_unique = df_final.groupby(['ID_Loja', 'Year','UF']).agg({'Ticket Médio': 'mean',
    'UnidadesVendidas': 'sum',
    'ValorTotal': 'sum'})
df_pivot = df_grouped_unique.reset_index().pivot_table(index=['ID_Loja', 'UF'], columns='Year', values=['Ticket Médio', 'UnidadesVendidas', 'ValorTotal'])


df_sorted_2023 = df_sorted.loc[(slice(None), 2023), ['Rank_Ticket_Medio']]
df_sorted_2023_sorted = df_sorted_2023.sort_values(by='Rank_Ticket_Medio')
id_loja_list = df_sorted_2023_sorted.index.get_level_values('ID_Loja').tolist()


df_pivot_reindexed = df_pivot.reindex(id_loja_list, level='ID_Loja')


import matplotlib.pyplot as plt
import numpy as np

# Calculate the variation for each column
for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    df_pivot_reindexed[(col, 'Variação 2022-2023')] = ((df_pivot_reindexed[(col, 2023)] - df_pivot_reindexed[(col, 2022)]) / df_pivot_reindexed[(col, 2022)]) * 100

# Fill NaN values with 0
df_pivot_reindexed = df_pivot_reindexed.fillna(0)

# Reindex the columns
df_pivot_reindexed = df_pivot_reindexed.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)

# Define the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Add the table to the figure
table_data = df_pivot_reindexed.reset_index().round(2).values.tolist()
column_labels = df_pivot_reindexed.reset_index().columns.tolist()

# Create a color dictionary for each 'ID_Loja'
id_loja_colors = {id_loja: colors[i % len(colors)] for i, id_loja in enumerate(df_pivot_reindexed.index.get_level_values('ID_Loja').unique())}

# Create a color matrix based on 'ID_Loja'
cell_colors = [[id_loja_colors[row[0]] for _ in row] for row in table_data]

# Highlight the largest numbers in each column, skipping the first two columns
for i in range(2, len(table_data[0])):
    column_data = [row[i] for row in table_data]
    max_value = max(column_data)
    for j in range(len(table_data)):
        if table_data[j][i] == max_value:
            cell_colors[j][i] = 'red'  # Change 'red' to any color you want

# Add the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center', cellColours=cell_colors)

# Adjust the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)

# Increase the font size
table.set_fontsize(25)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)

# Adjust the font size of 'ID_Loja' and 'UF' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:
        cell.set_fontsize(18)

# Hide the axes
ax.axis('off')

# Save the figure as .png
plt.savefig('df_pivot_reindexed_table.png')



In [ ]:
df_grouped_unique = df_final.groupby(['ID_Loja', 'Year','ClasseSocial']).agg({'Ticket Médio': 'mean',
    'UnidadesVendidas': 'sum',
    'ValorTotal': 'sum'})
df_pivot = df_grouped_unique.reset_index().pivot_table(index=['ID_Loja', 'ClasseSocial'], columns='Year', values=['Ticket Médio', 'UnidadesVendidas', 'ValorTotal'])


df_sorted_2023 = df_sorted.loc[(slice(None), 2023), ['Rank_Ticket_Medio']]
df_sorted_2023_sorted = df_sorted_2023.sort_values(by='Rank_Ticket_Medio')
id_loja_list = df_sorted_2023_sorted.index.get_level_values('ID_Loja').tolist()


df_pivot_reindexed = df_pivot.reindex(id_loja_list, level='ID_Loja')


import matplotlib.pyplot as plt
import numpy as np

# Calculate the variation for each column
for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    df_pivot_reindexed[(col, 'Variação 2022-2023')] = ((df_pivot_reindexed[(col, 2023)] - df_pivot_reindexed[(col, 2022)]) / df_pivot_reindexed[(col, 2022)]) * 100

# Fill NaN values with 0
df_pivot_reindexed = df_pivot_reindexed.fillna(0)

# Reindex the columns
df_pivot_reindexed = df_pivot_reindexed.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)

# Define the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Add the table to the figure
table_data = df_pivot_reindexed.reset_index().round(2).values.tolist()
column_labels = df_pivot_reindexed.reset_index().columns.tolist()

# Create a color dictionary for each 'ID_Loja'
id_loja_colors = {id_loja: colors[i % len(colors)] for i, id_loja in enumerate(df_pivot_reindexed.index.get_level_values('ID_Loja').unique())}

# Create a color matrix based on 'ID_Loja'
cell_colors = [[id_loja_colors[row[0]] for _ in row] for row in table_data]

# Highlight the largest numbers in each column, skipping the first two columns
for i in range(2, len(table_data[0])):
    column_data = [row[i] for row in table_data]
    max_value = max(column_data)
    for j in range(len(table_data)):
        if table_data[j][i] == max_value:
            cell_colors[j][i] = 'red'  # Change 'red' to any color you want

# Add the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center', cellColours=cell_colors)

# Adjust the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)

# Increase the font size
table.set_fontsize(25)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)

# Adjust the font size of 'ID_Loja' and 'ClasseSocial' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:
        cell.set_fontsize(18)

# Hide the axes
ax.axis('off')

# Save the figure as .png
plt.savefig('df_pivot_reindexed_table.png')



In [ ]:
df_grouped_unique = df_final.groupby(['ID_Loja', 'Year','Genero']).agg({'Ticket Médio': 'mean',
    'UnidadesVendidas': 'sum',
    'ValorTotal': 'sum'})
df_pivot = df_grouped_unique.reset_index().pivot_table(index=['ID_Loja', 'Genero'], columns='Year', values=['Ticket Médio', 'UnidadesVendidas', 'ValorTotal'])


df_sorted_2023 = df_sorted.loc[(slice(None), 2023), ['Rank_Ticket_Medio']]
df_sorted_2023_sorted = df_sorted_2023.sort_values(by='Rank_Ticket_Medio')
id_loja_list = df_sorted_2023_sorted.index.get_level_values('ID_Loja').tolist()


df_pivot_reindexed = df_pivot.reindex(id_loja_list, level='ID_Loja')


import matplotlib.pyplot as plt
import numpy as np

# Calculate the variation for each column
for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    df_pivot_reindexed[(col, 'Variação 2022-2023')] = ((df_pivot_reindexed[(col, 2023)] - df_pivot_reindexed[(col, 2022)]) / df_pivot_reindexed[(col, 2022)]) * 100

# Fill NaN values with 0
df_pivot_reindexed = df_pivot_reindexed.fillna(0)

# Reindex the columns
df_pivot_reindexed = df_pivot_reindexed.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)

# Define the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Add the table to the figure
table_data = df_pivot_reindexed.reset_index().round(2).values.tolist()
column_labels = df_pivot_reindexed.reset_index().columns.tolist()

# Create a color dictionary for each 'ID_Loja'
id_loja_colors = {id_loja: colors[i % len(colors)] for i, id_loja in enumerate(df_pivot_reindexed.index.get_level_values('ID_Loja').unique())}

# Create a color matrix based on 'ID_Loja'
cell_colors = [[id_loja_colors[row[0]] for _ in row] for row in table_data]

# Highlight the largest numbers in each column, skipping the first two columns
for i in range(2, len(table_data[0])):
    column_data = [row[i] for row in table_data]
    max_value = max(column_data)
    for j in range(len(table_data)):
        if table_data[j][i] == max_value:
            cell_colors[j][i] = 'red'  # Change 'red' to any color you want

# Add the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center', cellColours=cell_colors)

# Adjust the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)

# Increase the font size
table.set_fontsize(25)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)

# Adjust the font size of 'ID_Loja' and 'Genero' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:
        cell.set_fontsize(18)

# Hide the axes
ax.axis('off')

# Save the figure as .png
plt.savefig('df_pivot_reindexed_table.png')



In [ ]:
df_grouped_unique = df_final.groupby(['ID_Loja', 'Year','FaixaIdade']).agg({'Ticket Médio': 'mean',
    'UnidadesVendidas': 'sum',
    'ValorTotal': 'sum'})
df_pivot = df_grouped_unique.reset_index().pivot_table(index=['ID_Loja', 'FaixaIdade'], columns='Year', values=['Ticket Médio', 'UnidadesVendidas', 'ValorTotal'])


df_sorted_2023 = df_sorted.loc[(slice(None), 2023), ['Rank_Ticket_Medio']]
df_sorted_2023_sorted = df_sorted_2023.sort_values(by='Rank_Ticket_Medio')
id_loja_list = df_sorted_2023_sorted.index.get_level_values('ID_Loja').tolist()


df_pivot_reindexed = df_pivot.reindex(id_loja_list, level='ID_Loja')


import matplotlib.pyplot as plt
import numpy as np

# Calculate the variation for each column
for col in ['Ticket Médio', 'ValorTotal', 'UnidadesVendidas']:
    df_pivot_reindexed[(col, 'Variação 2022-2023')] = ((df_pivot_reindexed[(col, 2023)] - df_pivot_reindexed[(col, 2022)]) / df_pivot_reindexed[(col, 2022)]) * 100

# Fill NaN values with 0
df_pivot_reindexed = df_pivot_reindexed.fillna(0)

# Reindex the columns
df_pivot_reindexed = df_pivot_reindexed.reindex([
    ('Ticket Médio', 2022),
    ('Ticket Médio', 2023),
    ('Ticket Médio', 'Variação 2022-2023'),
    ('UnidadesVendidas', 2022),
    ('UnidadesVendidas', 2023),
    ('UnidadesVendidas', 'Variação 2022-2023'),
    ('ValorTotal', 2022),
    ('ValorTotal', 2023),
    ('ValorTotal', 'Variação 2022-2023')
], axis=1)

# Define the figure size
fig, ax = plt.subplots(figsize=(15, 10))

# Add the table to the figure
table_data = df_pivot_reindexed.reset_index().round(2).values.tolist()
column_labels = df_pivot_reindexed.reset_index().columns.tolist()

# Create a color dictionary for each 'ID_Loja'
id_loja_colors = {id_loja: colors[i % len(colors)] for i, id_loja in enumerate(df_pivot_reindexed.index.get_level_values('ID_Loja').unique())}

# Create a color matrix based on 'ID_Loja'
cell_colors = [[id_loja_colors[row[0]] for _ in row] for row in table_data]

# Highlight the largest numbers in each column, skipping the first two columns
for i in range(2, len(table_data[0])):
    column_data = [row[i] for row in table_data]
    max_value = max(column_data)
    for j in range(len(table_data)):
        if table_data[j][i] == max_value:
            cell_colors[j][i] = 'red'  # Change 'red' to any color you want

# Add the table to the figure
table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center', cellColours=cell_colors)

# Adjust the font size
table.auto_set_font_size(False)
table.set_fontsize(16)

# Increase the overall width and height of the table
table.scale(5, 4.5)

# Increase the font size
table.set_fontsize(25)

# Adjust the font size of the headers
for key, cell in table.get_celld().items():
    if key[0] == 0:
        cell.set_fontsize(20)

# Adjust the font size of 'ID_Loja' and 'FaixaIdade' columns
for key, cell in table.get_celld().items():
    if key[1] in [0, 1]:
        cell.set_fontsize(18)

# Hide the axes
ax.axis('off')

# Save the figure as .png
plt.savefig('df_pivot_reindexed_table.png')



##### Quinta parte da análise:
- Teste

###### nesta quinta parte da análise é possível verificar que:
O valor de p é 0,1850463335809991. 

Um limite comum de significância é 0,05. Se o valor de p for menor que 0,05, rejeitamos a hipótese nula e concluímos que há uma associação significativa entre as variáveis. Se o valor p for maior que 0,05, não rejeitamos a hipótese nula e concluímos que não temos evidências suficientes para afirmar que há uma associação significativa.

Nesse caso, o valor de p é maior que 0,05, portanto, não rejeitamos a hipótese nula. Isso significa que, com base nesse teste, não temos evidências suficientes para afirmar que há uma associação significativa entre "Grupo" e "Comprou".




In [ ]:
import openpyxl
import pandas as pd

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de Cliente
ws = wb['Cliente']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_c = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_c.columns:
    # Se o tipo de dados da coluna é object (string)
    if pd.api.types.is_object_dtype(df_c[col]):
        # Remova os espaços em branco no início e no final
        df_c[col] = df_c[col].str.strip()

In [ ]:
import openpyxl
import pandas as pd

# arquivo
wb = openpyxl.load_workbook('Exercicio_Python_CRM .xlsx', read_only=True)

# aba de Teste
ws = wb['Teste']

# tamanho do chunk para leitura de bases grandes
chunk_size = 100000
chunks = []

# cabeçalho (nomes das colunas) da primeira linha
header = [cell.value for cell in ws[1]]

# dados em chunks para bases com milhões de linhas
for i in range(2, ws.max_row, chunk_size):  # Comecça de 2 porque a primeira linha é o cabeçalho
    rows = ws[i:i + chunk_size]
    data = [[cell.value for cell in row] for row in rows]
    df = pd.DataFrame(data, columns=header)  # Usando o cabeçalho como nomes das colunas
    chunks.append(df)

# Concatenando os chunks em um único DataFrame
df_tst = pd.concat(chunks, axis=0)

# Para cada coluna no DataFrame
for col in df_tst.columns:
    # Se o tipo de dados da coluna é object (string)
    if pd.api.types.is_object_dtype(df_tst[col]):
        # Remova os espaços em branco no início e no final
        df_tst[col] = df_tst[col].str.strip()

In [ ]:
df_c.info()

In [ ]:
df_c = df_c.dropna(axis=1, how='all')

In [ ]:
df_c[df_c['Genero'].isnull()]

In [ ]:
for col in df_c.columns:
    print(f'Column: {col}')
    print(df_c[col].value_counts(dropna=False))

In [ ]:
df_tst.head(10)

In [ ]:
non_conforming_rows = df_tst.loc[
    ((df_tst['Enviado'] == 0) & (df_tst['Aberto'] != 0)) |
    ((df_tst['Enviado'] == 0) & (df_tst['Sessao'] != 0)) |
    ((df_tst['Recebido'] == 0) & (df_tst['Aberto'] != 0)) |
    ((df_tst['Aberto'] == 0) & (df_tst['Sessao'] != 0))
]

In [ ]:
non_conforming_rows

In [ ]:
df_tst = df_tst.drop(non_conforming_rows.index)

In [ ]:
df_tst.info()

In [ ]:
df_merged = df_tst.merge(df_c, on='CustomerID', how='left')

In [ ]:
df_merged

In [ ]:
from scipy.stats import chi2_contingency

# Create contingency table
contingency_table = pd.crosstab(df_merged['Grupo'], df_merged['Comprou'])

# Perform Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

In [ ]:
from scipy.stats import fisher_exact

# Create contingency table
contingency_table = pd.crosstab(df_merged['Grupo'], df_merged['Comprou'])

# Perform Fisher's Exact Test
odds_ratio, p_value = fisher_exact(contingency_table)

print(f"Odds Ratio: {odds_ratio}")
print(f"P-value: {p_value}")


# Se p < 0.05, há significância estatística
if p < 0.05:
    print("Existe relação significativa entre Grupo e Comprou")
else:
    print("Não há relação significativa entre Grupo e Comprou")


In [ ]:
import statsmodels.api as sm
from scipy import stats


# Criando a tabela de contingência
tabela_contingencia = pd.crosstab(df_merged['Grupo'], df_merged['Comprou'])

# Realizando o teste
chi2, p, df, expected = stats.chi2_contingency(tabela_contingencia)

# Imprimindo os resultados
print("Valor de p:", p)

# Se p < 0.05, há significância estatística
if p < 0.05:
    print("Existe relação significativa entre Grupo e Comprou")
else:
    print("Não há relação significativa entre Grupo e Comprou")


O teste qui-quadrado é um teste estatístico para determinar se há uma associação significativa entre duas variáveis categóricas. Nesse caso, as duas variáveis são "Grupo" e "Comprou".

A estatística do qui-quadrado é 1,7566226569244048. Esse valor representa a diferença entre as frequências observadas na tabela de contingência e as frequências que esperaríamos ver se o "Grupo" e o "Comprou" fossem independentes um do outro.

O valor de p é 0,1850463335809991. Essa é a probabilidade de observar uma estatística qui-quadrado tão ou mais extrema do que a estatística observada sob a hipótese nula de que o "Grupo" e o "Comprou" são independentes. 

Um limite comum de significância é 0,05. Se o valor de p for menor que 0,05, rejeitamos a hipótese nula e concluímos que há uma associação significativa entre as variáveis. Se o valor p for maior que 0,05, não rejeitamos a hipótese nula e concluímos que não temos evidências suficientes para afirmar que há uma associação significativa.

Nesse caso, o valor de p é maior que 0,05, portanto, não rejeitamos a hipótese nula. Isso significa que, com base nesse teste, não temos evidências suficientes para afirmar que há uma associação significativa entre "Grupo" e "Comprou".

A estatística qui-quadrado em si não indica inerentemente um resultado "bom" ou "ruim". É uma medida da diferença entre as frequências observadas e esperadas nas categorias de suas variáveis. Uma estatística de qui-quadrado mais alta significa uma diferença maior entre as frequências observadas e esperadas, o que pode sugerir uma relação mais forte entre as variáveis.

No entanto, o fato de essa diferença ser estatisticamente significativa é determinado pelo valor p. No seu caso, o valor p é 0,185, que é maior do que o nível de significância comum de 0,05. Isso significa que não rejeitamos a hipótese nula e concluímos que não temos evidências suficientes para afirmar que há uma associação significativa entre "Grupo" e "Comprou". 

Portanto, embora a estatística qui-quadrado forneça algumas informações sobre a relação entre as variáveis, o valor p é a chave para interpretar os resultados do teste.

Traduzido com a versão gratuita do tradutor - DeepL.com

O teste exato de Fisher é usado para determinar se há associações não aleatórias entre duas variáveis categóricas, semelhante ao teste qui-quadrado. 

A razão de chances é uma medida do tamanho do efeito, descrevendo a força da associação ou não-independência entre dois valores de dados binários. Uma razão de chances de 1 implica que o evento é igualmente provável em ambos os grupos. Uma razão de chances maior que 1 implica que o evento é mais provável no primeiro grupo. Uma razão de chances menor que 1 implica que o evento é menos provável no primeiro grupo. No seu caso, a razão de chances é 1,95, o que sugere que o evento (presumivelmente "Comprou" = 1) tem cerca de 1,95 vezes mais probabilidade de ocorrer no primeiro grupo em comparação com o segundo grupo.

O valor p é uma medida da probabilidade de que uma diferença observada possa ter ocorrido apenas por acaso. Quanto menor o valor p, maior a importância estatística da diferença observada. O valor p pode ser usado em testes de hipóteses para ajudá-lo a apoiar ou rejeitar a hipótese nula. Ele representa a probabilidade de que os resultados do seu teste ocorreram ao acaso. Se o valor p for menor que 0,05 (5%), ele é estatisticamente significativo. Isso significa que rejeitamos a hipótese nula (que afirma que não há efeito ou relação). Se o valor p for maior que 0,05, não é estatisticamente significativo. No seu caso, o valor p é 0,14, que é maior que 0,05, portanto, não rejeitamos a hipótese nula. Isso significa que, com base nesse teste, não temos evidências suficientes para dizer que há uma associação significativa entre "Grupo" e "Comprou".

Traduzido com a versão gratuita do tradutor - DeepL.com